In [2]:
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import torch
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from torch.utils.tensorboard import SummaryWriter
from sklearn.manifold import TSNE
from torch.optim.lr_scheduler import StepLR
import time
import torch.nn as nn
from datetime import datetime

In [3]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.ColorJitter()
])

full_dataset = datasets.ImageFolder('/kaggle/input/animal30/mammals', transform=transform)
class_counts = {class_name: 0 for class_name in full_dataset.classes}
for _, index in full_dataset.samples:
    class_name = full_dataset.classes[index]
    class_counts[class_name] += 1
print("Total number of classes:", len(full_dataset.classes))
print("Class names:", full_dataset.classes)
print("Number of images per class:")
for class_name, count in class_counts.items():
    print(f" - {class_name}: {count}")

Total number of classes: 30
Class names: ['african_elephant', 'alpaca', 'american_bison', 'anteater', 'arctic_fox', 'armadillo', 'baboon', 'badger', 'brown_bear', 'camel', 'giraffe', 'groundhog', 'highland_cattle', 'horse', 'jackal', 'kangaroo', 'koala', 'mongoose', 'mountain_goat', 'opossum', 'orangutan', 'polar_bear', 'porcupine', 'red_panda', 'rhinoceros', 'weasel', 'wildebeest', 'wombat', 'yak', 'zebra']
Number of images per class:
 - african_elephant: 347
 - alpaca: 333
 - american_bison: 343
 - anteater: 299
 - arctic_fox: 315
 - armadillo: 331
 - baboon: 330
 - badger: 310
 - brown_bear: 300
 - camel: 254
 - giraffe: 305
 - groundhog: 309
 - highland_cattle: 311
 - horse: 303
 - jackal: 278
 - kangaroo: 317
 - koala: 319
 - mongoose: 287
 - mountain_goat: 328
 - opossum: 330
 - orangutan: 340
 - polar_bear: 356
 - porcupine: 321
 - red_panda: 329
 - rhinoceros: 274
 - weasel: 282
 - wildebeest: 307
 - wombat: 315
 - yak: 254
 - zebra: 272


In [6]:
train_size = int(0.8 * len(full_dataset))
test_validation_size = len(full_dataset) - train_size
validation_size = test_validation_size // 2
test_size = test_validation_size - validation_size

train_dataset, test_validation_dataset = random_split(full_dataset, [train_size, test_validation_size])
validation_dataset, test_dataset = random_split(test_validation_dataset, [validation_size, test_size])

print("Size of the entire Dataset: ", len(full_dataset))
print("Size of the training Dataset: ", len(train_dataset))
print("Size of the validation Dataset: ", len(validation_dataset))
print("Size of the test Dataset: ", len(test_dataset))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

Size of the entire Dataset:  9299
Size of the training Dataset:  7439
Size of the validation Dataset:  930
Size of the test Dataset:  930


In [13]:
resnet18 = models.resnet18(pretrained=False)
num_classes = 30
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18.to(device)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.009
optimizer = torch.optim.SGD(resnet18.parameters(), lr=learning_rate, momentum=0.9)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

simple_name = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
writer = SummaryWriter(f'runs/{simple_name}')
model_save_path = 'resnet18_best_model.pth'

In [14]:
patience = 5
best_val_loss = np.inf
patience_counter = 0
epochs = 25
for epoch in range(epochs):
    epoch_start_time = time.time()
    train_loss = 0.0
    train_correct = 0
    total_train = 0
    resnet18.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet18(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / total_train
    writer.add_scalar('Loss/Train', train_loss / len(train_loader), epoch)
    writer.add_scalar('Accuracy/Train', train_accuracy, epoch)

    resnet18.eval()
    val_loss = 0.0
    val_correct = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = resnet18(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / total_val
    writer.add_scalar('Loss/Validation', val_loss / len(validation_loader), epoch)
    writer.add_scalar('Accuracy/Validation', val_accuracy, epoch)

    if val_loss / len(validation_loader) < best_val_loss:
        print(f"Validation loss decreased ({best_val_loss:.6f} --> {val_loss / len(validation_loader):.6f}). Saving model ...")
        best_val_loss = val_loss / len(validation_loader)
        torch.save(resnet18.state_dict(), model_save_path)
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break
    scheduler.step()
    print(f"Epoch {epoch+1}: Train Loss: {train_loss / len(train_loader):.4f}, "
      f"Train Accuracy: {train_accuracy:.2f}%, "
      f"Val Loss: {val_loss / len(validation_loader):.4f}, "
      f"Val Accuracy: {val_accuracy:.2f}%")
writer.close()

Validation loss decreased (inf --> 2.840727). Saving model ...
Epoch 1: Train Loss: 3.0963, Train Accuracy: 15.11%, Val Loss: 2.8407, Val Accuracy: 18.71%
Epoch 2: Train Loss: 2.5722, Train Accuracy: 27.41%, Val Loss: 2.8631, Val Accuracy: 23.76%
Validation loss decreased (2.840727 --> 2.376374). Saving model ...
Epoch 3: Train Loss: 2.3090, Train Accuracy: 33.67%, Val Loss: 2.3764, Val Accuracy: 34.62%
Validation loss decreased (2.376374 --> 2.241691). Saving model ...
Epoch 4: Train Loss: 2.1216, Train Accuracy: 38.42%, Val Loss: 2.2417, Val Accuracy: 39.57%
Epoch 5: Train Loss: 1.9494, Train Accuracy: 43.47%, Val Loss: 2.3566, Val Accuracy: 34.73%
Validation loss decreased (2.241691 --> 1.603815). Saving model ...
Epoch 6: Train Loss: 1.5146, Train Accuracy: 56.19%, Val Loss: 1.6038, Val Accuracy: 55.38%
Validation loss decreased (1.603815 --> 1.601449). Saving model ...
Epoch 7: Train Loss: 1.4012, Train Accuracy: 59.67%, Val Loss: 1.6014, Val Accuracy: 54.62%
Validation loss decre